In [1]:
import json
import os
import re
import torch
from mdutils.fileutils import MarkDownFile
from zipfile import ZipFile
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Normalize, Pooling, Transformer

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Run the three lines below
model_id = "hkunlp/instructor-large"
model = SentenceTransformer(model_id, cache_folder="cache_folder")
folder_path = "sentence-transformer-torchscript"

In [4]:
# 2. Go to cache_folder/hkunlp_instructor-large/1_Pooling/config.json 
# and remove "pooling_mode_weightedmean_tokens": false" & "pooling_mode_lasttoken": false
# as follows:
# {
#   "word_embedding_dimension": 768,
#   "pooling_mode_cls_token": false,
#   "pooling_mode_mean_tokens": true,
#   "pooling_mode_max_tokens": false,
#   "pooling_mode_mean_sqrt_len_tokens": false
# }

# 3. Run the function below so that you have model_zip_file saved at sentence-transformer-torchscript/instructor-large.zip
def save_as_pt(
    model,
    model_id,
    traced_folder,
    sentences: [str],
) -> str:
    model_name = str(model_id.split("/")[-1] + ".pt")

    model_path = os.path.join(traced_folder, model_name)
    save_json_folder_path = traced_folder
    model_output_path = traced_folder
    zip_file_name = str(model_id.split("/")[-1] + ".zip")
    zip_file_path = os.path.join(model_output_path, zip_file_name)

    if model.tokenizer.model_max_length > model.get_max_seq_length():
        model.tokenizer.model_max_length = model.get_max_seq_length()
        print(
            f"The model_max_length is not properly defined in tokenizer_config.json. Setting it to be {model.tokenizer.model_max_length}"
        )

    # save tokenizer.json in save_json_folder_name
    model.save(save_json_folder_path)


    # convert to pt format will need to be in cpu,
    # set the device to cpu, convert its input_ids and attention_mask in cpu and save as .pt format
    device = torch.device("cpu")
    cpu_model = model.to(device)
    features = cpu_model.tokenizer(
        sentences, return_tensors="pt", padding=True, truncation=True
    ).to(device)

    compiled_model = torch.jit.trace(
        cpu_model,
        (
            {
                "input_ids": features["input_ids"],
                "attention_mask": features["attention_mask"],
            }
        ),
        strict=False,
    )
    torch.jit.save(compiled_model, model_path)
    print("model file is saved to ", model_path)

    # zip model file along with tokenizer.json as output
    with ZipFile(str(zip_file_path), "w") as zipObj:
        zipObj.write(
            model_path,
            arcname=str(model_name),
        )
        zipObj.write(
            os.path.join(save_json_folder_path, "tokenizer.json"),
            arcname="tokenizer.json",
        )
    print("zip file is saved to ", zip_file_path, "\n")
    return zip_file_path

save_as_pt(model=model, model_id=model_id, traced_folder=folder_path, sentences=["for example providing a small sentence", "we can add multiple sentences"])

model file is saved to  sentence-transformer-torchscript/instructor-large.pt
zip file is saved to  sentence-transformer-torchscript/instructor-large.zip 



'sentence-transformer-torchscript/instructor-large.zip'

In [5]:
# 4. Run the function below so that you have model_config_json_file saved at sentence-transformer-torchscript/ml-commons_model_config.json
def _get_model_description_from_readme_file(model_id, readme_file_path) -> str:
    readme_data = MarkDownFile.read_file(readme_file_path)

    # Find the description section
    start_str = f"\n# {model_id}"
    start = readme_data.find(start_str)
    if start == -1:
        model_name = model_id.split("/")[1]
        start_str = f"\n# {model_name}"
        start = readme_data.find(start_str)
    end = readme_data.find("\n#", start + len(start_str))

    # If we cannot find the scope of description section, raise error.
    if start == -1 or end == -1:
        assert False, "Cannot find description in README.md file"

    # Parse out the description section
    description = readme_data[start + len(start_str) + 1 : end].strip()
    description = description.split("\n")[0]

    # Remove hyperlink and reformat text
    description = re.sub(r"\(.*?\)", "", description)
    description = re.sub(r"[\[\]]", "", description)
    description = re.sub(r"\*", "", description)

    # Remove unnecessary part if exists (i.e. " For an introduction to ...")
    # (Found in https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1/blob/main/README.md)
    unnecessary_part = description.find(" For an introduction to")
    if unnecessary_part != -1:
        description = description[:unnecessary_part]

    return description

def _generate_default_model_description(embedding_dimension) -> str:
    """
    Generate default model description of the model based on embedding_dimension

    ::param embedding_dimension: Embedding dimension of the model.
    :type embedding_dimension: int
    :return: Description of the model
    :rtype: string
    """
    print(
            "Using default description from embedding_dimension instead (You can overwrite this by specifying description parameter in make_model_config_json function"
    )
    description = f"This is a sentence-transformers model: It maps sentences & paragraphs to a {embedding_dimension} dimensional dense vector space."
    return description
    
def make_model_config_json(
    model,
    model_id,
    folder_path,
    version_number: str = 1,
    model_format: str = "TORCH_SCRIPT",
    embedding_dimension: int = None,
    pooling_mode: str = None,
    normalize_result: bool = None,
    description: str = None,
    all_config: str = None,
    model_type: str = None,
) -> str:
    config_json_file_path = os.path.join(folder_path, "config.json")
    model_name = model_id

    if (
        model_type is None
        or embedding_dimension is None
        or pooling_mode is None
        or normalize_result is None
    ):
        try:
            if embedding_dimension is None:
                embedding_dimension = model.get_sentence_embedding_dimension()

            for str_idx, module in model._modules.items():
                if model_type is None and isinstance(module, Transformer):
                    model_type = module.auto_model.__class__.__name__
                    model_type = model_type.lower().rstrip("model")
                elif pooling_mode is None and isinstance(module, Pooling):
                    pooling_mode = module.get_pooling_mode_str().upper()
                elif normalize_result is None and isinstance(module, Normalize):
                        normalize_result = True
            if normalize_result is None:
                normalize_result = False
        except Exception as e:
            raise Exception(
                f"Raised exception while getting model data from pre-trained hugging-face model object: {e}"
            )

    if description is None:
        readme_file_path = os.path.join(folder_path, "README.md")
        if os.path.exists(readme_file_path):
            try:
                description = _get_model_description_from_readme_file(
                    model_id,
                    readme_file_path
                )
            except Exception as e:
                print(f"Cannot scrape model description from README.md file: {e}")
                description = _generate_default_model_description(
                    embedding_dimension
                )
        else:
            print("Cannot find README.md file to scrape model description")
            description = _generate_default_model_description(
                embedding_dimension
            )

    if all_config is None:
        if not os.path.exists(config_json_file_path):
            raise Exception(
                str(
                    "Cannot find config.json in"
                    + config_json_file_path
                    + ". Please check the config.son file in the path."
                )
            )
        try:
            with open(config_json_file_path) as f:
                config_content = json.load(f)
                if all_config is None:
                    all_config = config_content
        except IOError:
            print(
                "Cannot open in config.json file at ",
                config_json_file_path,
                ". Please check the config.json ",
                "file in the path.",
            )

    model_config_content = {
        "name": model_name,
        "version": version_number,
        "description": description,
        "model_format": model_format,
        "model_task_type": "TEXT_EMBEDDING",
        "model_config": {
            "model_type": model_type,
            "embedding_dimension": embedding_dimension,
            "framework_type": "sentence_transformers",
            "pooling_mode": pooling_mode,
            "normalize_result": normalize_result,
            "all_config": json.dumps(all_config),
        },
    }


    model_config_file_path = os.path.join(
        folder_path, "ml-commons_model_config.json"
    )
    os.makedirs(os.path.dirname(model_config_file_path), exist_ok=True)
    with open(model_config_file_path, "w") as file:
        json.dump(model_config_content, file)
    print(
        "ml-commons_model_config.json file is saved at : ", model_config_file_path
    )

    return model_config_file_path

make_model_config_json(model=model, model_id=model_id, folder_path=folder_path, model_format='TORCH_SCRIPT')

ml-commons_model_config.json file is saved at :  sentence-transformer-torchscript/ml-commons_model_config.json


'sentence-transformer-torchscript/ml-commons_model_config.json'

In [6]:
# 5. Use 'sentence-transformer-torchscript/instructor-large.zip' and 'sentence-transformer-torchscript/ml-commons_model_config.json' to register the model to OpenSearch cluster

In [7]:
# import json
# import os
# import re
# import torch
# from mdutils.fileutils import MarkDownFile
# from zipfile import ZipFile
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.models import Normalize, Pooling, Transformer


# # 1. Run the three lines below
# model_id = "hkunlp/instructor-large"
# model = SentenceTransformer(model_id, cache_folder="cache_folder")
# folder_path = "sentence-transformer-torchscript"


# # 2. Go to cache_folder/hkunlp_instructor-large/1_Pooling/config.json 
# # and remove "pooling_mode_weightedmean_tokens": false" & "pooling_mode_lasttoken": false
# # as follows:
# # {
# #   "word_embedding_dimension": 768,
# #   "pooling_mode_cls_token": false,
# #   "pooling_mode_mean_tokens": true,
# #   "pooling_mode_max_tokens": false,
# #   "pooling_mode_mean_sqrt_len_tokens": false
# # }


# # 3. Run the function below so that you have model_zip_file saved at sentence-transformer-torchscript/instructor-large.zip
# def save_as_pt(
#     model,
#     model_id,
#     traced_folder,
#     sentences: [str],
# ) -> str:
#     model_name = str(model_id.split("/")[-1] + ".pt")

#     model_path = os.path.join(traced_folder, model_name)
#     save_json_folder_path = traced_folder
#     model_output_path = traced_folder
#     zip_file_name = str(model_id.split("/")[-1] + ".zip")
#     zip_file_path = os.path.join(model_output_path, zip_file_name)

#     if model.tokenizer.model_max_length > model.get_max_seq_length():
#         model.tokenizer.model_max_length = model.get_max_seq_length()
#         print(
#             f"The model_max_length is not properly defined in tokenizer_config.json. Setting it to be {model.tokenizer.model_max_length}"
#         )

#     # save tokenizer.json in save_json_folder_name
#     model.save(save_json_folder_path)


#     # convert to pt format will need to be in cpu,
#     # set the device to cpu, convert its input_ids and attention_mask in cpu and save as .pt format
#     device = torch.device("cpu")
#     cpu_model = model.to(device)
#     features = cpu_model.tokenizer(
#         sentences, return_tensors="pt", padding=True, truncation=True
#     ).to(device)

#     compiled_model = torch.jit.trace(
#         cpu_model,
#         (
#             {
#                 "input_ids": features["input_ids"],
#                 "attention_mask": features["attention_mask"],
#             }
#         ),
#         strict=False,
#     )
#     torch.jit.save(compiled_model, model_path)
#     print("model file is saved to ", model_path)

#     # zip model file along with tokenizer.json as output
#     with ZipFile(str(zip_file_path), "w") as zipObj:
#         zipObj.write(
#             model_path,
#             arcname=str(model_name),
#         )
#         zipObj.write(
#             os.path.join(save_json_folder_path, "tokenizer.json"),
#             arcname="tokenizer.json",
#         )
#     print("zip file is saved to ", zip_file_path, "\n")
#     return zip_file_path

# save_as_pt(model=model, model_id=model_id, traced_folder=folder_path, sentences=["for example providing a small sentence", "we can add multiple sentences"])


# # 4. Run the function below so that you have model_config_json_file saved at sentence-transformer-torchscript/ml-commons_model_config.json
# def _get_model_description_from_readme_file(model_id, readme_file_path) -> str:
#     readme_data = MarkDownFile.read_file(readme_file_path)

#     # Find the description section
#     start_str = f"\n# {model_id}"
#     start = readme_data.find(start_str)
#     if start == -1:
#         model_name = model_id.split("/")[1]
#         start_str = f"\n# {model_name}"
#         start = readme_data.find(start_str)
#     end = readme_data.find("\n#", start + len(start_str))

#     # If we cannot find the scope of description section, raise error.
#     if start == -1 or end == -1:
#         assert False, "Cannot find description in README.md file"

#     # Parse out the description section
#     description = readme_data[start + len(start_str) + 1 : end].strip()
#     description = description.split("\n")[0]

#     # Remove hyperlink and reformat text
#     description = re.sub(r"\(.*?\)", "", description)
#     description = re.sub(r"[\[\]]", "", description)
#     description = re.sub(r"\*", "", description)

#     # Remove unnecessary part if exists (i.e. " For an introduction to ...")
#     # (Found in https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1/blob/main/README.md)
#     unnecessary_part = description.find(" For an introduction to")
#     if unnecessary_part != -1:
#         description = description[:unnecessary_part]

#     return description

# def _generate_default_model_description(embedding_dimension) -> str:
#     """
#     Generate default model description of the model based on embedding_dimension

#     ::param embedding_dimension: Embedding dimension of the model.
#     :type embedding_dimension: int
#     :return: Description of the model
#     :rtype: string
#     """
#     print(
#             "Using default description from embedding_dimension instead (You can overwrite this by specifying description parameter in make_model_config_json function"
#     )
#     description = f"This is a sentence-transformers model: It maps sentences & paragraphs to a {embedding_dimension} dimensional dense vector space."
#     return description
    
# def make_model_config_json(
#     model,
#     model_id,
#     folder_path,
#     version_number: str = 1,
#     model_format: str = "TORCH_SCRIPT",
#     embedding_dimension: int = None,
#     pooling_mode: str = None,
#     normalize_result: bool = None,
#     description: str = None,
#     all_config: str = None,
#     model_type: str = None,
# ) -> str:
#     config_json_file_path = os.path.join(folder_path, "config.json")
#     model_name = model_id

#     if (
#         model_type is None
#         or embedding_dimension is None
#         or pooling_mode is None
#         or normalize_result is None
#     ):
#         try:
#             if embedding_dimension is None:
#                 embedding_dimension = model.get_sentence_embedding_dimension()

#             for str_idx, module in model._modules.items():
#                 if model_type is None and isinstance(module, Transformer):
#                     model_type = module.auto_model.__class__.__name__
#                     model_type = model_type.lower().rstrip("model")
#                 elif pooling_mode is None and isinstance(module, Pooling):
#                     pooling_mode = module.get_pooling_mode_str().upper()
#                 elif normalize_result is None and isinstance(module, Normalize):
#                         normalize_result = True
#             if normalize_result is None:
#                 normalize_result = False
#         except Exception as e:
#             raise Exception(
#                 f"Raised exception while getting model data from pre-trained hugging-face model object: {e}"
#             )

#     if description is None:
#         readme_file_path = os.path.join(folder_path, "README.md")
#         if os.path.exists(readme_file_path):
#             try:
#                 description = _get_model_description_from_readme_file(
#                     model_id,
#                     readme_file_path
#                 )
#             except Exception as e:
#                 print(f"Cannot scrape model description from README.md file: {e}")
#                 description = _generate_default_model_description(
#                     embedding_dimension
#                 )
#         else:
#             print("Cannot find README.md file to scrape model description")
#             description = _generate_default_model_description(
#                 embedding_dimension
#             )

#     if all_config is None:
#         if not os.path.exists(config_json_file_path):
#             raise Exception(
#                 str(
#                     "Cannot find config.json in"
#                     + config_json_file_path
#                     + ". Please check the config.son file in the path."
#                 )
#             )
#         try:
#             with open(config_json_file_path) as f:
#                 config_content = json.load(f)
#                 if all_config is None:
#                     all_config = config_content
#         except IOError:
#             print(
#                 "Cannot open in config.json file at ",
#                 config_json_file_path,
#                 ". Please check the config.json ",
#                 "file in the path.",
#             )

#     model_config_content = {
#         "name": model_name,
#         "version": version_number,
#         "description": description,
#         "model_format": model_format,
#         "model_task_type": "TEXT_EMBEDDING",
#         "model_config": {
#             "model_type": model_type,
#             "embedding_dimension": embedding_dimension,
#             "framework_type": "sentence_transformers",
#             "pooling_mode": pooling_mode,
#             "normalize_result": normalize_result,
#             "all_config": json.dumps(all_config),
#         },
#     }


#     model_config_file_path = os.path.join(
#         folder_path, "ml-commons_model_config.json"
#     )
#     os.makedirs(os.path.dirname(model_config_file_path), exist_ok=True)
#     with open(model_config_file_path, "w") as file:
#         json.dump(model_config_content, file)
#     print(
#         "ml-commons_model_config.json file is saved at : ", model_config_file_path
#     )

#     return model_config_file_path

# make_model_config_json(model=model, model_id=model_id, folder_path=folder_path, model_format='TORCH_SCRIPT')


# # 5. Use 'sentence-transformer-torchscript/instructor-large.zip' and 'sentence-transformer-torchscript/ml-commons_model_config.json' to register the model to OpenSearch cluster

In [10]:
#!pip install opensearch-py opensearch-py-ml

import os
import sys
sys.path.append(os.path.abspath(os.path.join('../../..')))

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="TracerWarning: torch.tensor")
warnings.filterwarnings("ignore", message="using SSL with verify_certs=False is insecure.")

import opensearch_py_ml as oml
from opensearchpy import OpenSearch
from opensearch_py_ml.ml_models import SentenceTransformerModel
# import mlcommon to later register the model to OpenSearch Cluster
from opensearch_py_ml.ml_commons import MLCommonClient

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

client = get_os_client()

# Connect to ml_common client with OpenSearch client
ml_client = MLCommonClient(client)

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [12]:
ml_client.register_model("sentence-transformer-torchscript/instructor-large.zip", "sentence-transformer-torchscript/ml-commons_model_config.json", isVerbose=True)

Total number of chunks 135
Sha1 value of the model file:  eb3be3c144a3564f666b8c19bc5c4e841b4c352422e2150e439e2076a6c5806b
Model meta data was created successfully. Model Id:  _cu7cYoBSA5PdoWsvUMh
uploading chunk 1 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 10 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 11 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 12 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 13 of 135
Model id: {'status': 'Uploaded'}
uploading chunk 14 of 135
Model id: {'status'

Task ID: _su8cYoBSA5PdoWsfUOb
Model deployed successfully


'_cu7cYoBSA5PdoWsvUMh'

In [17]:
import numpy as np


sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
input_sentences = [[sentence, instruction]]


In [42]:
embedding_output_torch = ml_client.generate_embedding("ZE6tcYoBmvm98tEoUR17", input_sentences)

In [21]:
len(embedding_output_torch['inference_results'][0]['output'][0]['data'])

768

In [37]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings)

load INSTRUCTOR_Transformer
max_seq_length  512
[[-6.15552627e-02  1.04199909e-02  5.88440849e-03  1.93768777e-02
   5.71417809e-02  2.57655680e-02 -4.01848811e-05 -2.80044544e-02
  -2.92965453e-02  4.91884835e-02  6.78200126e-02  2.18692459e-02
   4.54528630e-02  1.50187174e-02 -4.84451912e-02 -3.25259753e-02
  -3.56492661e-02  1.19935432e-02 -6.83915243e-03  3.03126313e-02
   5.17491624e-02  3.48140560e-02  4.91032703e-03  6.68928474e-02
   1.52824204e-02  3.54217105e-02  1.07743740e-02  6.89828917e-02
   4.44019511e-02 -3.23419496e-02  1.24267889e-02 -2.15528179e-02
  -1.62690915e-02 -4.15058285e-02 -2.42290483e-03 -3.07159079e-03
   4.27047350e-02  1.56428684e-02  2.57813130e-02  5.92843294e-02
  -1.99174136e-02  1.32361799e-02  1.08407950e-02 -4.00610566e-02
  -1.36212725e-03 -1.57032814e-02 -2.53812242e-02 -1.31972805e-02
  -7.83779379e-03 -1.14009008e-02 -4.82025407e-02 -2.58416273e-02
  -4.98771109e-03  4.98239547e-02  1.19490065e-02 -5.55060543e-02
  -2.82120351e-02 -3.3220887

In [38]:
embeddings.shape

(1, 768)

In [26]:
original_pre_trained_model = model
original_embedding_data = list(
    original_pre_trained_model.encode(input_sentences, convert_to_numpy=True)
)

In [27]:
for i in range(len(input_sentences)):
    print(i)
    print(np.testing.assert_allclose(embeddings[i], embedding_output_torch['inference_results'][i]['output'][0]['data'], rtol=1e-03, atol=1e-05))

0


AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 766 / 768 (99.7%)
Max absolute difference: 0.0982903
Max relative difference: 6898.21135732
 x: array([-4.475518e-02, -3.724144e-04,  6.277785e-03,  2.341405e-02,
        4.438463e-02,  1.630010e-02, -1.171885e-02, -5.577739e-03,
       -3.750417e-02,  2.511203e-02,  4.287542e-02,  6.856018e-03,...
 y: array([-5.717582e-02,  2.493422e-03, -3.901535e-03,  1.850889e-02,
        5.135028e-02,  2.320276e-02,  6.400365e-03, -1.590035e-02,
       -2.774814e-02,  4.663043e-02,  6.030675e-02,  2.010986e-02,...

In [ ]:
loaded_model = torch.jit.load("traced_sentence_transformer.pt")
loaded_model.eval()

In [ ]:
POST /_plugins/_ml/_predict/text_embedding/{model_id}
{
    "text_docs": [[question A, answer A], [question B, answer B], [question C, answer C]],
    "return_number": true,
    "target_response": [ "sentence_embedding" ]
}

In [15]:
ml_client._client

<OpenSearch([{'host': 'localhost', 'port': 9200, 'use_ssl': True}])>

In [18]:
input_sentences

[['3D ActionSLAM: wearable person tracking in multi-floor environments',
  'Represent the Science title:']]

In [23]:
API_BODY = {"text_docs": input_sentences}

In [24]:
ml_client._client.transport.perform_request(
            method="POST",
            url=f"/_plugins/_ml/_predict/text_embedding/_cu7cYoBSA5PdoWsvUMh",
            body=API_BODY,
        )

TransportError: TransportError(500, 'null_pointer_exception', 'Cannot invoke "org.opensearch.ml.common.input.MLInput.setAlgorithm(org.opensearch.ml.common.FunctionName)" because "mlInput" is null')

In [28]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 1024, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)

In [31]:
hg_model = SentenceTransformer(model_id, cache_folder="cache_folder")

In [43]:
hg_embed = hg_model.encode(input_sentences)

In [34]:
input_sentences

[['3D ActionSLAM: wearable person tracking in multi-floor environments',
  'Represent the Science title:']]

In [44]:
for i in range(len(input_sentences)):
    print(i)
    print(np.testing.assert_allclose(embeddings, hg_embed, rtol=1e-03, atol=1e-05))

0


AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 765 / 768 (99.6%)
Max absolute difference: 0.02885431
Max relative difference: 285.67206
 x: array([[-6.155526e-02,  1.041999e-02,  5.884408e-03,  1.937688e-02,
         5.714178e-02,  2.576557e-02, -4.018488e-05, -2.800445e-02,
        -2.929655e-02,  4.918848e-02,  6.782001e-02,  2.186925e-02,...
 y: array([[-6.028877e-02,  2.991482e-03, -4.056946e-04,  2.204840e-02,
         5.370862e-02,  1.982622e-02, -3.396538e-04, -1.533841e-02,
        -3.740350e-02,  4.192707e-02,  6.052603e-02,  1.625427e-02,...